In [1]:
import guidance
gpt = guidance.llms.OpenAI("gpt-4")  # or gpt-3.5-turbo (though 3.5 tends to follow instructions less well)

This example shows two methods of soliciting user input. First, for user inputs that don't depend on GPT results we can simply inject them as variables into the program when it's first created. The `topic` is an example of that.

For later-stage user inputs which may depend on GPT generations, we can use the `await` approach which tells `guidance` to [pause execution](https://github.com/microsoft/guidance#pausing-execution-with-await) until the new input is provided. In the example below, the `user_answers` variable follows this approach because in order to prompt the user for answers, we need to have the model-generated questions.

By combining these two techniques, we can generate an entire gpt-based program in a single definition.

In [2]:
create_quiz = guidance('''
{{#system~}}
You are a helpful and terse assistant that generates and grades quizzes. You produce only outputs, no banter.
{{~/system}}

{{#user~}}
Please create a quiz on the following topic:
{{topic}}
List {{num_learning_objectives}} potential learning objectives.
Do not output a quiz yet.
{{~/user}}

{{#assistant~}}
{{gen 'learning_objectives' temperature=.75 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please provide a bullet list of {{num_questions}} short-answer questions for the quiz. (no newlines within the question)
Questions should be aligned with learning objectives.
Do not write the answers yet.
{{~/user}}

{{#assistant~}}
{{gen 'questions' temperature=0 max_tokens=100}}
{{~/assistant}}

{{#user~}}
Finally, output a numbered list of grading rubrics describing how to award points
(hitting all requirements will award {{points_per_question}} points per question).
Rubrics should help a non-expert grade assign grades.
Rubrics must fit on a single line (no newlines within the rubric).
{{~/user}}

{{#assistant~}}
{{gen 'rubric' temperature=0 max_tokens=1000}}
{{~/assistant}}

{{#user~}}
Please grade the following quiz answers.

{{await 'user_answers'}}

After generating a score, provide helpful and constructive feedback for the student.
{{~/user}}

{{#assistant~}}
{{gen 'report' temperature=.75 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Finally, please advise how the student could improve on the overall learning objectives.
{{~/user}}

{{#assistant~}}
{{gen 'advice' temperature=.75 max_tokens=300}}
{{~/assistant}}
''', llm=gpt)

In [3]:
quiz_topic = input("What is the topic of your quiz?") or "computer science"

What is the topic of your quiz?seasons in Seattle


In [4]:
my_quiz = create_quiz(**{
    "topic": quiz_topic,
    "num_learning_objectives": 1,
    "num_questions": 2,
    "points_per_question": 5
})

Stop program system You are a helpful and terse assistant that generates and grades quizzes. You produce only outputs, no banter. user Please create a quiz on the following topic:
 seasons in Seattle 
List 1 potential learning objectives.
Do not output a quiz yet. assistant Learning Objective: Understand the characteristics and weather patterns of each season in Seattle. user Great, now please provide a bullet list of 2 short-answer questions for the quiz. (no newlines within the question)
Questions should be aligned with learning objectives.
Do not write the answers yet. assistant 1. Describe the typical weather patterns and characteristics of Seattle during the spring season.
2. How does the amount of rainfall in Seattle vary between the summer and winter seasons? user Finally, output a numbered list of grading rubrics describing how to award points
(hitting all requirements will award 5 points per question).
Rubrics should help a non-expert grade assign grades.
Rubrics must fit on a single line (no newlines within the rubric). assistant 1. (0-5 points) Award points based on the accuracy and completeness of the description of Seattle's spring weather patterns and characteristics; 5 points for a thorough and accurate description.
2. (0-5 points) Award points based on the accuracy and clarity of the comparison between summer and winter rainfall in Seattle; 5 points for a clear and accurate comparison. user Please grade the following quiz answers.

 {{await 'user_answers'}} 

After generating a score, provide helpful and constructive feedback for the student. {{#assistant~}} 
 {{gen 'report' temperature=.75 max_tokens=300}} 
 {{~/assistant}} 

 {{#user~}} 
Finally, please advise how the student could improve on the overall learning objectives.
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'advice' temperature=.75 max_tokens=300}} 
 {{~/assistant}}

In [5]:
# split the questions output (they are generated as one per line)
questions = [q for q in my_quiz["questions"].split("\n") if q]

user_answers = ""
for idx, question in enumerate(questions):
    user_answer = input(question) or "I don't know"
    user_answers += f"{idx+1}. {user_answer}\n"

1. Describe the typical weather patterns and characteristics of Seattle during the spring season.rain and clouds are common, with some dry and warm days sprinkled in
2. How does the amount of rainfall in Seattle vary between the summer and winter seasons?summer is very dry, sometimes approaching drought conditions. winter tends to be very wet and cloudy with some snow but rain is more common nearly every day


In [6]:
my_quiz = my_quiz(user_answers=user_answers)

Stop program system You are a helpful and terse assistant that generates and grades quizzes. You produce only outputs, no banter. user Please create a quiz on the following topic:
 seasons in Seattle 
List 1 potential learning objectives.
Do not output a quiz yet. assistant Learning Objective: Understand the characteristics and weather patterns of each season in Seattle. user Great, now please provide a bullet list of 2 short-answer questions for the quiz. (no newlines within the question)
Questions should be aligned with learning objectives.
Do not write the answers yet. assistant 1. Describe the typical weather patterns and characteristics of Seattle during the spring season.
2. How does the amount of rainfall in Seattle vary between the summer and winter seasons? user Finally, output a numbered list of grading rubrics describing how to award points
(hitting all requirements will award 5 points per question).
Rubrics should help a non-expert grade assign grades.
Rubrics must fit on a single line (no newlines within the rubric). assistant 1. (0-5 points) Award points based on the accuracy and completeness of the description of Seattle's spring weather patterns and characteristics; 5 points for a thorough and accurate description.
2. (0-5 points) Award points based on the accuracy and clarity of the comparison between summer and winter rainfall in Seattle; 5 points for a clear and accurate comparison. user Please grade the following quiz answers.

 1. rain and clouds are common, with some dry and warm days sprinkled in
2. summer is very dry, sometimes approaching drought conditions. winter tends to be very wet and cloudy with some snow but rain is more common nearly every day
 

After generating a score, provide helpful and constructive feedback for the student. assistant Score: 9/10

Feedback:
1. (4/5 points) Your answer provides a general overview of Seattle's spring weather. For a more thorough description, consider mentioning average temperatures and the transition from winter to summer weather patterns.
2. (5/5 points) Excellent comparison of Seattle's summer and winter rainfall. You clearly explained the differences between the two seasons, mentioning drought conditions in summer and wet and cloudy weather in winter. Keep up the good work! user Finally, please advise how the student could improve on the overall learning objectives. assistant To improve on the overall learning objectives, the student should:

1. Research more about the average temperatures and weather patterns during each season in Seattle, as this will provide a deeper understanding of the characteristics of the city's climate.
2. Explore any specific weather-related events or phenomena that may occur in Seattle across different seasons, such as "Pineapple Express" or "the Big Dark."
3. Compare Seattle's seasons with other cities in the Pacific Northwest or across the United States to gain a better understanding of regional weather patterns.
4. Study how Seattle's climate and seasonal changes impact its flora, fauna, and human activities, such as outdoor recreation and agriculture.

By expanding their knowledge in these areas, the student will gain a more comprehensive understanding of the seasons in Seattle and their various implications.